In [226]:
# Import the basic spark library

from pyspark.sql import SparkSession

# Create an entry point to the PySpark Application
spark = SparkSession.builder \
      .master("local") \
      .appName("schemaBook") \
      .getOrCreate()
# master contains the URL of your remote spark instance or 'local'

In [227]:
#schema book
df_book = spark.read.option("multiline","true").json("book-db-manual.json")
df_book.printSchema()
#df_book.show()

root
 |-- author: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- orcid: string (nullable = true)
 |-- citations: long (nullable = true)
 |-- crossref: string (nullable = true)
 |-- editor: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- orcid: string (nullable = true)
 |-- ee: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- isbn: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- key: string (nullable = true)
 |-- keyword: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- pages: long (nullable = true)
 |-- publisher: string (nullable = true)
 |-- series: struct (nullable = true)
 |    |-- href: string (nullable = true)
 |    |-- title: string (nullable = true)
 |-- title: string (nullable = true)
 |-- volume: long (nullable = true)
 |-- 

In [228]:
#schema article
df_article = spark.read.option("multiline","true").json("article-db-manual_conrefs.json")
df_article.printSchema()
#df_article.show()

root
 |-- author: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- orcid: string (nullable = true)
 |-- citations: long (nullable = true)
 |-- cite: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ee: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- journal: string (nullable = true)
 |-- key: string (nullable = true)
 |-- keyword: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- note: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- pages: long (nullable = true)
 |-- publisher: string (nullable = true)
 |-- title: string (nullable = true)
 |-- volume: long (nullable = true)
 |-- year: long (nullable = true)



In [229]:
#schema incollection
df_incollection = spark.read.option("multiline","true").json("incollection-db-manual.json")
df_incollection.printSchema()
#df_incollection.show()

root
 |-- author: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- orcid: string (nullable = true)
 |-- citations: long (nullable = true)
 |-- crossref: string (nullable = true)
 |-- ee: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- key: string (nullable = true)
 |-- keyword: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- pages: long (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



3) Removal of all the publications by a specific author

In [230]:
#union
df_pub=df_book.unionByName(df_article,allowMissingColumns=True).unionByName(df_incollection,allowMissingColumns=True)
df_pub.printSchema()
#df_pub.show()

root
 |-- author: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- orcid: string (nullable = true)
 |-- citations: long (nullable = true)
 |-- crossref: string (nullable = true)
 |-- editor: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- orcid: string (nullable = true)
 |-- ee: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- isbn: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- key: string (nullable = true)
 |-- keyword: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- pages: long (nullable = true)
 |-- publisher: string (nullable = true)
 |-- series: struct (nullable = true)
 |    |-- href: string (nullable = true)
 |    |-- title: string (nullable = true)
 |-- title: string (nullable = true)
 |-- volume: long (nullable = true)
 |-- 

In [231]:
from pyspark.sql.functions import array_contains, col, explode

df_pub_key=df_pub.select(col("key"), explode(df_pub.author)).withColumnRenamed("col", "author").select(col("key"),"author.name","author.orcid").filter(col("author.orcid")=="Jennifer Widom").select(col("key")).withColumnRenamed("key", "key_k")
df_pub_key.show(truncate=False)

+-----------------------------------+
|key_k                              |
+-----------------------------------+
|journals/debu/HammerGWLZ95         |
|journals/debu/Widom92              |
|journals/debu/ChawatheGW94         |
|journals/debu/Widom99              |
|journals/dr/Widom99a               |
|journals/dr/Widom99c               |
|journals/dr/Widom99b               |
|journals/dr/Widom99                |
|journals/sigmod/SnodgrassHMOPRRWY98|
|journals/tods/AikenHW95            |
|journals/tkde/Widom96              |
+-----------------------------------+



In [232]:

df_pub=df_pub.join(df_pub_key, df_pub.key==df_pub_key.key_k, "leftouter").filter(df_pub_key.key_k.isNull()).drop(df_pub_key.key_k).select(col("key"), col("author"))
df_pub.show(truncate=False)

+--------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|key                       |author                                                                                                                                                                                                                                             |
+--------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|reference/sp/2015ci       |null                                                                                                                                                     

In [233]:
#check
df_pub.select(col("key"), explode(df_pub.author)).withColumnRenamed("col", "author").select(col("key"), "author.name", "author.orcid").filter(col("author.name")=="Jennifer Widom").show(truncate=False)

+---+----+-----+
|key|name|orcid|
+---+----+-----+
+---+----+-----+

